# STOCK CLOSING PRICE PREDICTION

In [17]:
#importing libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import pandas_datareader as pdr

key="30c0973b8a648106ae38faea9031b9a3924c7469"

In [18]:
#Getting Dataset
df=pdr.get_data_tiingo('DIS', api_key=key)

C:\Users\sneha\AppData\Local\Temp\ipykernel_7192\3742500115.py:2: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df=pdr.get_data_tiingo('DIS', api_key=key)


In [19]:
df.to_csv('DIS.csv')

In [20]:
df=pd.read_csv('DIS.csv', date_parser=True)

In [21]:
# df.head()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,DIS,2018-11-30 00:00:00+00:00,115.49,117.065,115.06,116.84,16857777,113.205388,114.749232,112.783894,114.528683,16857777,0.00,1.0
1,DIS,2018-12-03 00:00:00+00:00,115.74,116.920,114.91,116.65,9272917,113.450443,114.607100,112.636862,114.342441,9272917,0.00,1.0
2,DIS,2018-12-04 00:00:00+00:00,112.87,116.290,112.66,115.62,10806737,110.637217,113.989563,110.431371,113.332816,10806737,0.00,1.0
3,DIS,2018-12-06 00:00:00+00:00,114.33,114.340,111.16,112.06,10546187,112.068335,112.078137,108.961044,109.843240,10546187,0.00,1.0
4,DIS,2018-12-07 00:00:00+00:00,111.98,113.990,111.61,113.86,8305465,110.627415,112.613136,110.261884,112.484706,8305465,0.88,1.0


In [22]:
df.tail()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
1250,DIS,2023-11-20 00:00:00+00:00,95.03,95.5850,93.870,93.98,14114766,95.03,95.5850,93.870,93.98,14114766,0.0,1.0
1251,DIS,2023-11-21 00:00:00+00:00,94.45,94.9600,93.960,94.73,7654843,94.45,94.9600,93.960,94.73,7654843,0.0,1.0
1252,DIS,2023-11-22 00:00:00+00:00,95.07,95.7061,94.760,94.91,7330932,95.07,95.7061,94.760,94.91,7330932,0.0,1.0
1253,DIS,2023-11-24 00:00:00+00:00,96.06,96.5100,94.900,94.90,5440468,96.06,96.5100,94.900,94.90,5440468,0.0,1.0
1254,DIS,2023-11-27 00:00:00+00:00,95.17,95.7400,94.515,95.74,11049470,95.17,95.7400,94.515,95.74,11049470,0.0,1.0


## Data Pre-processing

In [23]:
df['date'] = pd.to_datetime(df['date'])
# df['date'] = df['date'].dt.date
df

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,DIS,2018-11-30 00:00:00+00:00,115.49,117.0650,115.060,116.84,16857777,113.205388,114.749232,112.783894,114.528683,16857777,0.00,1.0
1,DIS,2018-12-03 00:00:00+00:00,115.74,116.9200,114.910,116.65,9272917,113.450443,114.607100,112.636862,114.342441,9272917,0.00,1.0
2,DIS,2018-12-04 00:00:00+00:00,112.87,116.2900,112.660,115.62,10806737,110.637217,113.989563,110.431371,113.332816,10806737,0.00,1.0
3,DIS,2018-12-06 00:00:00+00:00,114.33,114.3400,111.160,112.06,10546187,112.068335,112.078137,108.961044,109.843240,10546187,0.00,1.0
4,DIS,2018-12-07 00:00:00+00:00,111.98,113.9900,111.610,113.86,8305465,110.627415,112.613136,110.261884,112.484706,8305465,0.88,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,DIS,2023-11-20 00:00:00+00:00,95.03,95.5850,93.870,93.98,14114766,95.030000,95.585000,93.870000,93.980000,14114766,0.00,1.0
1251,DIS,2023-11-21 00:00:00+00:00,94.45,94.9600,93.960,94.73,7654843,94.450000,94.960000,93.960000,94.730000,7654843,0.00,1.0
1252,DIS,2023-11-22 00:00:00+00:00,95.07,95.7061,94.760,94.91,7330932,95.070000,95.706100,94.760000,94.910000,7330932,0.00,1.0
1253,DIS,2023-11-24 00:00:00+00:00,96.06,96.5100,94.900,94.90,5440468,96.060000,96.510000,94.900000,94.900000,5440468,0.00,1.0


In [24]:
# Splitting train and test set
df_training=df[df['date']<='2022-01-01'].copy()
df_test=df[df['date']>'2022-01-01'].copy()
df_test, df_training

(     symbol                      date   close      high       low    open  \
 777     DIS 2022-01-03 00:00:00+00:00  156.76  157.5600  155.3600  155.83   
 778     DIS 2022-01-04 00:00:00+00:00  155.73  160.3200  155.5500  158.59   
 779     DIS 2022-01-05 00:00:00+00:00  155.19  159.3800  155.1000  156.52   
 780     DIS 2022-01-06 00:00:00+00:00  156.90  157.7700  153.6791  156.24   
 781     DIS 2022-01-07 00:00:00+00:00  157.83  159.3000  156.2900  156.90   
 ...     ...                       ...     ...       ...       ...     ...   
 1250    DIS 2023-11-20 00:00:00+00:00   95.03   95.5850   93.8700   93.98   
 1251    DIS 2023-11-21 00:00:00+00:00   94.45   94.9600   93.9600   94.73   
 1252    DIS 2023-11-22 00:00:00+00:00   95.07   95.7061   94.7600   94.91   
 1253    DIS 2023-11-24 00:00:00+00:00   96.06   96.5100   94.9000   94.90   
 1254    DIS 2023-11-27 00:00:00+00:00   95.17   95.7400   94.5150   95.74   
 
         volume  adjClose   adjHigh    adjLow  adjOpen  adjVol

In [25]:
training_df=df_training.drop(['symbol','date','adjClose','adjHigh','adjLow','adjOpen','adjVolume','divCash','splitFactor'], axis=1)
training_df

,close,high,low,open,volume
0,115.49,117.065,115.060,116.84,16857777
1,115.74,116.920,114.910,116.65,9272917
2,112.87,116.290,112.660,115.62,10806737
3,114.33,114.340,111.160,112.06,10546187
4,111.98,113.990,111.610,113.86,8305465
...,...,...,...,...,...
772,152.80,154.020,151.400,152.88,7765470
773,155.20,156.500,152.410,152.62,12198679
774,154.87,155.920,154.180,154.56,8737004
775,155.93,157.070,155.515,155.71,7228438


## Feature Scaling

In [26]:
scaler=MinMaxScaler(feature_range=(0,1))
training_df=scaler.fit_transform(training_df)
# training_df=scaler.fit_transform(np.array(training_df).reshape(-1,1))
training_df



array([[0.25596212, 0.25734405, 0.30937849, 0.2796145 , 0.16692474],
       [0.25811451, 0.25609124, 0.30808906, 0.27797225, 0.07736419],
       [0.23340508, 0.250648  , 0.28874753, 0.26906954, 0.09547524],
       ...,
       [0.59500646, 0.5930534 , 0.6456632 , 0.60564415, 0.07103623],
       [0.60413259, 0.60298946, 0.65713917, 0.61558408, 0.05322338],
       [0.59517865, 0.59866943, 0.65047709, 0.61696703, 0.04096006]])

## Converting into Dataset Matrix (taking previous 60 values)

In [27]:
X_train=[]
y_train=[]

In [28]:
training_df.shape

(777, 5)

In [29]:
ltrain=training_df.shape[0]

for i in range(60, ltrain):
    X_train.append(training_df[i-60:i])
    y_train.append(training_df[i,0])


In [30]:
X_train, y_train=np.array(X_train), np.array(y_train)
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 5))
X_train.shape, y_train.shape

((717, 60, 5), (717,))

## Building LSTM 

In [31]:
regressior=Sequential()
regressior.add(LSTM(units=60, activation='relu', return_sequences=True, input_shape=(X_train.shape[1],5)))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units=60, activation='relu', return_sequences=True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units=80, activation='relu', return_sequences=True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units=120, activation='relu'))
regressior.add(Dropout(0.2))

regressior.add(Dense(units=1))

regressior.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 60, 60)            15840     
                                                                 
 dropout_4 (Dropout)         (None, 60, 60)            0         
                                                                 
 lstm_5 (LSTM)               (None, 60, 60)            29040     
                                                                 
 dropout_5 (Dropout)         (None, 60, 60)            0         
                                                                 
 lstm_6 (LSTM)               (None, 60, 80)            45120     
                                                                 
 dropout_6 (Dropout)         (None, 60, 80)            0         
                                                                 
 lstm_7 (LSTM)               (None, 120)              

In [32]:
regressior.compile(optimizer='adam', loss='mean_squared_error')
regressior.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
12/12 [==============================] - 25s 496ms/step - loss: 0.2114
Epoch 2/50
12/12 [==============================] - 6s 487ms/step - loss: 0.0341
Epoch 3/50
12/12 [==============================] - 5s 406ms/step - loss: 0.0204
Epoch 4/50
12/12 [==============================] - 4s 315ms/step - loss: 0.0163
Epoch 5/50
12/12 [==============================] - 4s 359ms/step - loss: 0.0111
Epoch 6/50
12/12 [==============================] - 5s 436ms/step - loss: 0.0112
Epoch 7/50
12/12 [==============================] - 4s 362ms/step - loss: 0.0110
Epoch 8/50
12/12 [==============================] - 7s 562ms/step - loss: 0.0112
Epoch 9/50
12/12 [==============================] - 6s 467ms/step - loss: 0.0112
Epoch 10/50
12/12 [==============================] - 7s 575ms/step - loss: 0.0103
Epoch 11/50
12/12 [==============================] - 5s 411ms/step - loss: 0.0097
Epoch 12/50
12/12 [==============================] - 6s 447ms/step - loss: 0.0091
Epoch 13/50
12/12 [=====

## Preparing Test Dataset

In [33]:
df_test.head()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
777,DIS,2022-01-03 00:00:00+00:00,156.76,157.56,155.3600,155.83,10228257,156.76,157.56,155.3600,155.83,10228257,0.0,1.0
778,DIS,2022-01-04 00:00:00+00:00,155.73,160.32,155.5500,158.59,16582041,155.73,160.32,155.5500,158.59,16582041,0.0,1.0
779,DIS,2022-01-05 00:00:00+00:00,155.19,159.38,155.1000,156.52,12272097,155.19,159.38,155.1000,156.52,12272097,0.0,1.0
780,DIS,2022-01-06 00:00:00+00:00,156.90,157.77,153.6791,156.24,11095326,156.90,157.77,153.6791,156.24,11095326,0.0,1.0
781,DIS,2022-01-07 00:00:00+00:00,157.83,159.30,156.2900,156.90,9554622,157.83,159.30,156.2900,156.90,9554622,0.0,1.0


In [34]:
df_training.tail()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
772,DIS,2021-12-27 00:00:00+00:00,152.80,154.02,151.400,152.88,7765470,152.80,154.02,151.400,152.88,7765470,0.0,1.0
773,DIS,2021-12-28 00:00:00+00:00,155.20,156.50,152.410,152.62,12198679,155.20,156.50,152.410,152.62,12198679,0.0,1.0
774,DIS,2021-12-29 00:00:00+00:00,154.87,155.92,154.180,154.56,8737004,154.87,155.92,154.180,154.56,8737004,0.0,1.0
775,DIS,2021-12-30 00:00:00+00:00,155.93,157.07,155.515,155.71,7228438,155.93,157.07,155.515,155.71,7228438,0.0,1.0
776,DIS,2021-12-31 00:00:00+00:00,154.89,156.57,154.740,155.87,6189861,154.89,156.57,154.740,155.87,6189861,0.0,1.0


In [35]:
past_60_days=df_training.tail(60)
# past_60_days
test_df=past_60_days.append(df_test) #past 60 days taken for the first value of test data
test_df=test_df.drop(['symbol','date','adjClose','adjHigh','adjLow','adjOpen','adjVolume','divCash','splitFactor'], axis=1)
test_df

C:\Users\sneha\AppData\Local\Temp\ipykernel_7192\458701125.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df=past_60_days.append(df_test) #past 60 days taken for the first value of test data


,close,high,low,open,volume
717,177.71,179.6300,177.3300,177.70,7779997
718,176.74,178.6000,176.4029,177.93,4132182
719,173.52,176.5200,173.4800,176.02,8046270
720,173.13,174.4900,172.5450,173.62,5188380
721,172.96,173.7000,170.9400,173.63,8918152
...,...,...,...,...,...
1250,95.03,95.5850,93.8700,93.98,14114766
1251,94.45,94.9600,93.9600,94.73,7654843
1252,95.07,95.7061,94.7600,94.91,7330932
1253,96.06,96.5100,94.9000,94.90,5440468


In [36]:
scaler=MinMaxScaler(feature_range=(0,1))
test_df=scaler.fit_transform(test_df)
test_df


array([[1.        , 1.        , 1.        , 0.99005924, 0.05515183],
       [0.99014127, 0.98968308, 0.99059728, 0.99236871, 0.        ],
       [0.95741437, 0.96884891, 0.96095299, 0.97319008, 0.05917765],
       ...,
       [0.16007724, 0.15938239, 0.16256842, 0.15875088, 0.04836235],
       [0.17013924, 0.16743459, 0.16398831, 0.15865047, 0.01978016],
       [0.16109361, 0.15972194, 0.16008361, 0.16708505, 0.10458345]])

In [37]:
test_df.shape

(538, 5)

In [38]:
X_test=[]
y_test=[]

In [39]:
ltest=test_df.shape[0]
for i in range(60,ltest):
    X_test.append(test_df[i-60:i])
    y_test.append(test_df[i,0])

In [40]:
X_test, y_test=np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((478, 60, 5), (478,))

## Predicting the stock closing price

In [41]:
y_pred=regressior.predict(X_test)

15/15 [==============================] - 3s 83ms/step


In [42]:
y_pred=scaler.inverse_transform(y_pred)

ValueError: non-broadcastable output operand with shape (478,1) doesn't match the broadcast shape (478,5)

In [ ]:
import math
from sklearn.metrics import mean_squared_error

math.sqrt(mean_squared_error(y_test, y_pred))

0.06863727203119092

In [ ]:
y_test

array([7.87071857e-01, 7.76603313e-01, 7.71114951e-01, 7.88494766e-01,
       7.97946946e-01, 7.85445675e-01, 7.98556764e-01, 7.97642037e-01,
       7.73655859e-01, 7.38083139e-01, 7.41437138e-01, 7.19483687e-01,
       6.94176237e-01, 5.90100620e-01, 5.90913711e-01, 5.81258258e-01,
       5.51682082e-01, 5.70179896e-01, 6.02805163e-01, 6.46915337e-01,
       6.62364061e-01, 6.43358065e-01, 6.17034251e-01, 6.37259884e-01,
       6.42240065e-01, 6.41935156e-01, 6.90212420e-01, 7.40319138e-01,
       7.12978961e-01, 7.27004777e-01, 7.66338042e-01, 7.82904767e-01,
       7.48348409e-01, 7.32188231e-01, 6.98953146e-01, 6.76389877e-01,
       7.12267507e-01, 7.13588779e-01, 7.02713690e-01, 6.74662059e-01,
       6.91330420e-01, 6.73340787e-01, 6.24047159e-01, 5.50665718e-01,
       5.32879358e-01, 5.52190263e-01, 5.52088627e-01, 5.32879358e-01,
       5.05234272e-01, 5.57780262e-01, 5.97824982e-01, 6.11342616e-01,
       6.19778433e-01, 6.02703527e-01, 6.17847342e-01, 5.92743165e-01,
      

## Visualization the results

In [ ]:
import plotly.graph_objs as go

# Create a trace for actual values
trace_actual = go.Scatter(x=df_test['date'], y=y_test, mode='lines', name='Actual', line=dict(color='red'))

# Create a trace for predicted values
trace_pred = go.Scatter(x=df_test['date'], y=y_pred.flatten(), mode='lines', name='Predicted', line=dict(color='green'))

# Create a trace for markers with reduced size
trace_markers1 = go.Scatter(
    x=df_test['date'],
    y=y_test.flatten(),
    mode='markers',
    name='',
    showlegend=False,
    marker=dict(color='red', size=3),  # Adjust the size here
)

trace_markers2 = go.Scatter(
    x=df_test['date'],
    y=y_pred.flatten(),
    mode='markers',
    name='',
    showlegend=False,
    marker=dict(color='green', size=3),  # Adjust the size here
)

# Create the Plotly figure
fig = go.Figure(data=[trace_actual, trace_pred, trace_markers1, trace_markers2])

# Customize the layout
fig.update_layout(
    title='Stock Price Prediction',
    xaxis_title='Time',
    yaxis_title='Stock Price',
     width=800,
    height=600
)

# Show the Plotly figure
fig.show()


In [ ]:
# # Generate predictions for the next 10 days
# last_60_days = df1[-60:]  # Take the last 60 days from the scaled data
# X_next_10_days = []  # Initialize the input sequence for the next 10 days

# for i in range(10):
#     X_next_10_days.append(last_60_days[-60:])
#     last_60_days = np.concatenate([last_60_days[1:], y_pred[-1].reshape(1, 1)])

# X_next_10_days = np.array(X_next_10_days)

# # Reshape the input data for prediction
# X_next_10_days = np.reshape(X_next_10_days, (X_next_10_days.shape[0], X_next_10_days.shape[1], 5))

# # Predict the next 10 days
# y_next_10_days = regressior.predict(X_next_10_days)

# # Invert the scaling to get the predicted stock prices in the original scale
# y_next_10_days = scaler.inverse_transform(np.concatenate([last_60_days[-10:].reshape(-1, 1), y_next_10_days]))

# # Create a new DataFrame for the next 10 days with the predicted values and the corresponding dates
# next_10_days_dates = pd.date_range(start=df_test['date'].iloc[-1] + pd.Timedelta(days=1), periods=10)
# df_next_10_days = pd.DataFrame({'date': next_10_days_dates, 'predicted_price': y_next_10_days.flatten()})

# # Print the predicted stock prices for the next 10 days
# print(df_next_10_days)


In [ ]:
# df_past_60_days_for_future = df_test.tail(60).copy()
# df_past_60_days_for_future=df_past_60_days_for_future.drop(['symbol','date','adjClose','adjHigh','adjLow','adjOpen','adjVolume','divCash','splitFactor'], axis=1)
# df_past_60_days_for_future

# X_input=list(df_past_60_days_for_future)
# X_input = df_past_60_days_for_future.iloc[:, 0].tolist()
# X_input

# # demonstrate prediction for next 10 days
# from numpy import array
# import pandas as pd 

# lst_output=[]
# n_steps=60
# i=0

# while i < 10:
#     if len(X_input) > 60:
#         X_input = np.array(temp_input[1:])
#         print("{} day input {}".format(i, X_input))
#         X_input = X_input.reshape(1, n_steps, 1)
#         yhat = regressior.predict(X_input, verbose=0)
#         print("{} day output {}".format(i, yhat))
#         temp_input.extend(yhat[0].tolist())
#         temp_input = temp_input[1:]
#         lst_output.extend(yhat.tolist())
#         i = i + 1
#     else:
#         X_input = np.array(X_input).reshape((1, n_steps, 1))
#         yhat = regressior.predict(X_input, verbose=0)
#         print("{} day output {}".format(i, yhat))
#         temp_input.extend(yhat[0].tolist())
#         lst_output.extend(yhat.tolist())
#         i = i + 1

# print(lst_output)

# # day_new=np.arange(1,61)
# # day_pred=np.arange(61,71)
# # plt.plot(day_new,df_train['Close'],'b',label='Actual')
# # plt.plot(day_pred,lst_output,'r',label='Predicted')
# # plt.legend()
# # plt.show()




In [ ]:
# X_train.shape[1]
# print(len(temp_input))


In [ ]:
# df_past_60_days_for_future